In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install fastapi uvicorn pyngrok transformers==4.52.4 accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 81.4 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.1 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
).eval()

def generate_text(prompt, max_length=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2026-02-09 16:19:03.679620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770653944.065294      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770653944.172080      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770653945.108352      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770653945.108385      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770653945.108389      55 computation_placer.cc:177] computation placer alr

...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
NGROK_TOKEN="2f2QVaA9FMNmOr2S4CPZQ5HAbOL_5oQXCgxaTNXhxsQeZqNaD"
API_KEY="secret123"

In [7]:
from fastapi import FastAPI,Request,HTTPException
import uvicorn,threading,time,socket
from pyngrok import ngrok,conf
app=FastAPI()
@app.post("/generate")
async def gen(req: Request):
    if req.headers.get("authorization")!=f"Bearer {API_KEY}":
        raise HTTPException(status_code=401,detail="Unauthorized")
    data=await req.json()
    return {
        "response":generate_text(
            data.get("prompt",""),
            data.get("max_length",300)
        )
    }

In [9]:
def free_port():
    s=socket.socket()
    s.bind(('',0))
    port=s.getsockname()[1]
    s.close()
    return port

port=free_port()
conf.get_default().auth_token=NGROK_TOKEN
public_url=ngrok.connect(port).public_url
print(public_url)
def run(): uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

    

https://82e4-34-132-216-141.ngrok-free.app                                                          


INFO:     Started server process [55]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:56403 (Press CTRL+C to quit)


INFO:     102.45.48.84:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     102.45.48.84:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     102.45.48.84:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     102.45.48.84:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     102.45.48.84:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     102.45.48.84:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     102.45.48.84:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     102.45.48.84:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     102.45.48.84:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     102.45.48.84:0 - "POST /generate HTTP/1.1" 200 OK
